[Emscripten](https://emscripten.org/index.html) is a C/C++ compiler to wasm

# Installation

```shell
cd ~/local
git clone https://github.com/emscripten-core/emsdk.git
cd emsdk
./emsdk install latest
./emsdk activate latest
```

# Usage

Config environment for emscripten:

```shell
source ./emsdk_env.sh
```

Basic program:
```shelll
emcc main.c
node a.out.js
```

# Update

```shell
./emsdk update
./emsdk install latest
./emsdk activate latest
```

# Example: create wasm lib

Source file fact.c:
```c
int fact(int x)
{
  return x <= 1 ? 1 : x * fact(x-1);
}
```

Compile:
```shell
emcc fact.c -O1 -s SIDE_MODULE=1 -s EXPORTED_FUNCTIONS="['_fact']" -o fact.wasm
wasm-opt -O3 fact.wasm -o fact.wasm
```

Output of `wasm-dis fact.wsasm`:
```
(module
 (type $none_=>_none (func))
 (type $i32_=>_i32 (func (param i32) (result i32)))
 (global $global$0 i32 (i32.const 0))
 (export "__wasm_apply_relocs" (func $0))
 (export "fact" (func $1))
 (export "__dso_handle" (global $global$0))
 (export "__post_instantiate" (func $0))
 (func $0 (; 0 ;)
  (nop)
 )
 (func $1 (; 1 ;) (param $0 i32) (result i32)
  (if (result i32)
   (i32.ge_s
    (local.get $0)
    (i32.const 2)
   )
   (i32.mul
    (call $1
     (i32.add
      (local.get $0)
      (i32.const -1)
     )
    )
    (local.get $0)
   )
   (i32.const 1)
  )
 )
 ;; custom section "dylink", size 5
)
```

# Options

Options are set using ```-s OPTION1=VAL1 -s OPTION2=val2```.  
The whole list of options can be find [here](https://github.com/emscripten-core/emscripten/blob/1.29.12/src/settings.js)

## EXPORTED_FUNCTIONS

List of functions that must be exported, names prefixed with '_'.  
This may not be needed, it's just that withoutt this option LLVM optis may remove some functions that we would want exported.

## SIDE_MODULE=1

Create a dynamic library. Used to define a wasm library exporting only a few functions.

# Extra exports

when building, emscipten may export some extra symbols

## \_\_dso_handle

int global of value $0$, guard to identify dynamic shared objects

## \_\_post_instantiate

Used to run code when module is instantied.  
Usually to initialize global variables (run global constructors)

## \_\_wasm_apply_relocs

Called by \_\_post_instantiate, perform symbols relocation ?